# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from sklearn.metrics import mean_absolute_error
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
import requests
from matplotlib.ticker import MaxNLocator
from regression_bias_corrector import LinearBiasCorrector
from scipy.stats import mannwhitneyu, variation, levene
from copy import deepcopy


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# Load data

In [15]:
path = f"E:/YandexDisk/Work/bbd/mriya/08_AIJ_2025"

df = pd.read_excel(f"{path}/Результат.xlsx", index_col=0)

colors = {
    'Низкий': 'chartreuse',
    'Высокий': 'crimson'
}


fig, axs = plt.subplot_mosaic(
    [
        ['11', '12'],
    ],
    figsize=(6.6, 4),
    # height_ratios=[4],
    width_ratios=[3, 1.2],
    gridspec_kw={
        # "bottom": 0.14,
        # "top": 0.95,
        "wspace": 0.3,
        #"hspace": 0.5,
    },
)


xy_min = df[['Age', 'Биологический возраст']].min().min()
xy_max = df[['Age', 'Биологический возраст']].max().max()
xy_ptp = xy_max - xy_min
bisect = sns.lineplot(
    x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
    y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
    linestyle='--',
    color='black',
    linewidth=1.0,
    ax=axs['11']
)
scatter = sns.scatterplot(
    data=df,
    x='Age',
    y='Биологический возраст',
    hue='Риск ССЗ',
    palette=colors,
    linewidth=0.5,
    alpha=0.75,
    edgecolor="k",
    s=20,
    hue_order=list(colors.keys()),
    legend=True,
    ax=axs['11'],
)
axs['11'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
axs['11'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
axs['11'].set_xlabel('Хронологический возраст')

sns.violinplot(
    data=df,
    x='Риск ССЗ',
    y=f"Возрастная акселерация",
    hue='Риск ССЗ',
    palette=colors,
    density_norm='width',
    order=['Низкий', 'Высокий'],
    saturation=0.75,
    linewidth=1.0,
    ax=axs['12'],
    legend=False,
    cut=0,
)

_, mw_pval = mannwhitneyu(df.loc[df['Риск ССЗ'] == 'Низкий', "Возрастная акселерация"].values, df.loc[df['Риск ССЗ'] == 'Высокий', "Возрастная акселерация"].values, alternative='two-sided')
pval_formatted = [f'{mw_pval:.2e}']
annotator = Annotator(
    axs['12'],
    pairs=[('Низкий', 'Высокий')],
    data=df,
    x='Риск ССЗ',
    y='Возрастная акселерация',
    order=['Низкий', 'Высокий'],
)
annotator.set_custom_annotations(pval_formatted)
annotator.configure(loc='inside', verbose=0)
annotator.annotate()

fig.savefig(f"{path}/fig2.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/fig2.pdf", bbox_inches='tight')
plt.close(fig)

c:\Users\user\anaconda3\envs\py311\Lib\site-packages\statannotations\Annotator.py:825: UserWarning: Annotator was reconfigured without applying the test (again) which will probably lead to unexpected results
  warnings.warn("Annotator was reconfigured without applying the "
